## 主題:綜合所得稅申報核定資料

目的:https://www.fia.gov.tw/singlehtml/43?cntId=b93b91349217404d9d910ebe4e930fa0
爬取 107年度綜合所得稅申報核定統計專冊-HTML(.zip)
解壓縮後，針對 107_165-A.html ~ 107_165-Z.html 進行解析並存成一份整合CSV表格

In [1]:
#匯入相關套件
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep

import os
import pandas as pd
from bs4 import BeautifulSoup
import string
import zipfile

In [9]:
#爬蟲機
def crawler(url):
    #webDriver基本設定
    options=webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--incognito")
    options.add_argument("--disable-popup-blocking")
    driver=webdriver.Chrome(options=options)
    driver.set_window_size(1200,960)
    
    driver.get(url)
    driver.find_element(By.CSS_SELECTOR, 'a.file-zip[title="下載：107年度綜合所得稅申報核定統計專冊-HTML(.zip).zip"]').click()
    
    
#解壓縮機器
def unziper():
    html_list = []
    while True:
        if not os.path.exists('C:\\Users\\USER\\Downloads\\107年度綜合所得稅申報核定統計專冊-HTML(.zip).zip'):
            print("還未完成下載...")
            continue
        else:
            print("正在解壓縮zip")
            path = os.path.abspath(f'永豐作業\\1. 綜合所得稅所得\\107綜合所得稅html')
            z = zipfile.ZipFile('C:\\Users\\USER\\Downloads\\107年度綜合所得稅申報核定統計專冊-HTML(.zip).zip', 'r')
            for l in 'ABCDEFGHIJKMNOPQTUVWXZ':
                file = f'isa107/107_165-{l}.html'
                z.extract(file, path) #解壓縮特定檔案到指定路徑
            break


#html解析機
def html_parser(path: str) -> list:

    #路徑變數
    f = open(path, 'r', encoding="utf-8")
    
    #指定lxml作為解析器
    soup = BeautifulSoup(f.read(), 'lxml')    

    #計算tables總數
    tables = soup.find_all("table")
    tab_number = len(tables)

    #計算每個table中總共有多少tr
    tar = soup.find_all("tr")
    tar_number = len(tar)
    
    #欄位名稱
    tab = tables[1]
    columns = tab.find_all("tr")[5].get_text().strip('\n').split('\n')

    #補上縣市、鄉鎮市區、村里
    add_list = ['縣市', '鄉鎮市區', '村里']
    new_columns = add_list + columns

    #縣市別
    row = tab.find_all("tr")[3].get_text().strip('\n').split('\n')
    county = row[0][4:]
    county_list = [county] 
 
    #村里
    area_list = ''

    #一次列出所有資料List
    tables_list = []

    tables = soup.find_all("table")
    tab = tables[0]
    for n in range(0, (tar_number)):
        try:
            data = tab.find_all("tr")[n].get_text().strip('\n').split('\n')[0:]
            o = '其\u3000他'
            s = '合\u3000計'
            if len(data) == 10:
                area_list = [data[0]]
                if o in area_list:
                    area_list = ['其他']
                    data[1] = '其他'
                    data = data[1:10]
                    data = county_list + area_list + data
                    new_data = dict(zip(new_columns, data))
                    tables_list.append(new_data)
#                   print(new_data)
                    
                    
                else:
                    data = data[1:10]
                    data = county_list + area_list + data
                    new_data = dict(zip(new_columns, data))
                    tables_list.append(new_data)
#                   print(new_data)
        
            if len(data) == 9:

                if o in data[0]:
                    data[0] = '其他'
                    data = county_list + area_list + data
                    new_data = dict(zip(new_columns, data))
                    tables_list.append(new_data)
#                   print(new_data)
                        
                elif s in data[0]:
                    data[0] = '合計'
                    data = county_list + area_list + data
                    new_data = dict(zip(new_columns, data))
                    tables_list.append(new_data)
#                   print(new_data)
            
                else:
                    data = county_list + area_list + data
                    new_data = dict(zip(new_columns, data))
                    tables_list.append(new_data)
#                   print(new_data)
        except:
            continue
    f.close()
    return tables_list

In [10]:
# #執行爬蟲
# crawler('https://www.fia.gov.tw/singlehtml/43?cntId=b93b91349217404d9d910ebe4e930fa0')

# #解壓縮下載完成檔案
# unziper()

#套上檔案路徑並執行html解析程式
file_list = []
for l in 'ABCDEFGHIJKMNOPQTUVWXZ':
    path = os.path.abspath(f'永豐作業\\1. 綜合所得稅所得\\107綜合所得稅html\\isa107\\107_165-{l}.html')
    file_list = file_list + html_parser(path)
    print(path)

D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-A.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-B.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-C.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-D.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-E.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-F.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-G.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-H.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-I.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-J.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-K.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-M.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\isa107\107_165-N.html
D:\jupyter_notes\永豐銀行\永豐作業\1. 綜合所得稅所得\107綜合所得稅html\

In [20]:
#存成DataFrame
df = pd.DataFrame(file_list)
#去除字串中的whitespace
df['縣市'] = df['縣市'].str.replace(" ","")
df['鄉鎮市區'] = df['鄉鎮市區'].str.replace(" ","")
df

,縣市,鄉鎮市區,村里,納稅單位,綜合所得總額,平均數,中位數,第一分位數,第三分位數,標準差,變異係數
0,臺北市,松山區,中崙里,1412,2164630,1533,907,455,1802,3046.16,198.70
1,臺北市,松山區,自強里,3228,4498386,1394,841,413,1708,2685.42,192.70
2,臺北市,松山區,鵬程里,1852,2279390,1231,749,414,1514,1485.53,120.70
3,臺北市,松山區,東榮里,2924,4853375,1660,912,454,1904,3284.52,197.88
4,臺北市,松山區,介壽里,1555,2407651,1548,906,442,1889,3244.67,209.56
...,...,...,...,...,...,...,...,...,...,...,...
8545,連江縣,莒光鄉,合計,561,545756,973,756,477,1231,822.10,84.51
8546,連江縣,東引鄉,中柳村,214,212902,995,895,628,1240,605.68,60.88
8547,連江縣,東引鄉,樂華村,248,259342,1046,954,621,1296,646.65,61.84
8548,連江縣,東引鄉,其他,0,0,0,0,0,0,0.00,0.00


In [21]:
#存檔成CSV檔
df.to_csv(os.path.abspath('永豐作業\\1. 綜合所得稅所得\\my_answer\\107_村里所得.csv'),index=False, encoding="utf-8-sig")